# 3. 경계 검출 (Edge detection)
* **3.1 영상 이진화** : Simple/Adaptive/Otsu Thresholding
* **3.2 형상변환** : Erosion, Dilation, Opening, Closing
* **3.3 그래디언트** : Sobel/Scharr, Laplacian

## 3.1 영상 이진화 (Thresholding/Binarization)

### Simple Thresholding
* 픽셀의 값이 경계값을 넘을 경우 최대값으로 변롼함. 결과적으로 흑/백으로 변환함
* `cv2.threshold(원본이미지, 경계값, 최대값, 스타일)` 함수를 이용함
* 스타일 : `THRESH_BINARY`, `THRESH_BINARY_INV`, `THRESH_TRUNC`, `THRESH_TOZERO`, `THRESH_TOZERO_INV`
* 반환값1 : threashold값(otsu에서활용, 다른스타일의경우 입력한 경계값고정)
* 반환값2 : 경게값처리된 이미지
![threshold types](https://docs.opencv.org/4.0.1/threshold.png)

In [ ]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('images/black.jpg')
ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
ret,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
ret,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
ret,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
ret,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]

fig = plt.figure(figsize=(15,9))
for i in range(6): 
    ax = fig.add_subplot(2, 3, i+1)
    ax.imshow(images[i],'gray') 
    ax.set_title(titles[i]) 
    ax.axis('off')
    
plt.show()

### Adaptive Thresholding
* 하나이상의 경계값을 사용할 경우 효과가 좋음 (하나의 이미지에 다수의 조명상태가 존재하는경우)
* 영상을 구획별로 잘라서, 구획별로 다른 경계값을 적용함
* `cv2.threshold(원본이미지, 경계값, 최대값, 적응형스타일, 구획의이진화스타일, 구획크기, 가중치조정범위)`
* 적응형스타일 : `cv2.ADAPTIVE_THRESH_MEAN_C`(주변의평균값 사용), `cv2.ADAPTIVE_THRESH_GAUSSIAN_C`(주변에 가우시간가중치 적용한 합 사용)

In [ ]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('images/shadow_text.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(img, 5)

ret,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
titles = ['Original Image' \
          , 'Global Thresholding (v = 127)' \
          , 'Adaptive Mean Thresholding' \
          , 'Adaptive Gaussian Thresholding']

fig = plt.figure(figsize=(20,10))
images = [img, th1, th2, th3]
for i in range(4):
    ax = fig.add_subplot(2, 2, i+1)
    ax.imshow(images[i],'gray') 
    ax.set_title(titles[i]) 
    ax.axis('off')
    
plt.show()

### Otsu 이진화
* 이미지 히스토그램의 피크 지점이 두개인경우(bimodal image) 두 피크점 사이를 임계값으로 잡아줌
* `cv2.thresold()`함수의 첫번째 반환값이 조정된 임꼐값을 의미함
* **이진화적용전 가우시안블러를 적용할 경우 노이즈 제거 효과를 얻을 수 있음**

In [ ]:
%matplotlib inline
import cv2
import numpy as np 
from matplotlib import pyplot as plt 

img = cv2.imread('images/noise.png',0)

# 단순 이진화
ret1, th1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# OTSU 이진화
ret2, th2 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# 가우시안 블러 처리한 후 OTSU 이진화
blur = cv2.GaussianBlur(img,(5,5),0)
ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# 임계값 출력
print("Thresold 1 : ", ret1)
print("Thresold 2 : ", ret2)
print("Thresold 3 : ", ret3)

# plot all the images and their histograms
plt.figure(figsize=(16,12))
images = [img, 0, th1, img, 0, th2, blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)', 'Original Noisy Image','Histogram',"Otsu's Thresholding", 'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]

for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])

plt.show()


## 3.2 형상 변환(Morphological Transformations)

### Erosion
* 두꺼운 객체를 깍아내고 가지치기 하는 것
* 각 Pixel에 structuring element를 적용하여 하나라도 0이 있으면 대상 pixel을 제거하는 방법
* 커널 안에 0이 하나라도 있는 경우 0처리 해버리므로 **작은 노이즈가 삭제되는 효과**가 있음
![erosion](https://opencv-python.readthedocs.io/en/latest/_images/image01.png)

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

img = cv2.imread('images/sudoku.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret1, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# Erosion
kernel = np.ones((4,4),np.uint8)
erosion = cv2.erode(img,kernel,iterations = 1)


# 화면출력
plt.figure(figsize=(20,8))
images = [img, erosion]
titles = ['Original','Erosion']
for i in range(2):
    plt.subplot(1,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i]), plt.xticks([]), plt.yticks([])
plt.show()

### Dilation
* 객체의 몸집을 불리는 효과로, 자잘한 구멍이 있는 경우 메워지는 효과
* 곡선검출시 선이 중간에 끊어지는 것도 erosion을 이용해서 메꾸면 부드럽게 이어지는 효과를 볼 수 있음
* 대상 pixel에 대해서 OR 연산을 수행합니다. 즉 겹치는 부분이 하나라도 있으면 이미지를 확장
![dilation](https://opencv-python.readthedocs.io/en/latest/_images/image03.png)

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

img = cv2.imread('images/sudoku.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret1, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# Erosion
kernel = np.ones((4,4),np.uint8)
dilation = cv2.dilate(img,kernel,iterations = 1)

# 화면출력
plt.figure(figsize=(20,8))
images = [img, dilation]
titles = ['Original','Dilation']
for i in range(2):
    plt.subplot(1,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i]), plt.xticks([]), plt.yticks([])
plt.show()

### Opening & Closing
* Opening과 Closing은 Erosion과 Dilation의 조합 결과. 어느 것을 먼저 적용을 하는 차이
* **Opeing** : Erosion적용 후 Dilation 적용. 작은 Object나 돌기 제거에 적합
* **Closing** : Dilation적용 후 Erosion 적용. 전체적인 윤곽 파악에 적합

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

img = cv2.imread('images/sudoku.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret1, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# Kernel
kernel = np.ones((4,4),np.uint8)

# Opening : erode -> dilation (깍아내고 불림)
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

# Closing : dilation -> erode (불리고 깍아냄)
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

# 화면출력
plt.figure(figsize=(18,6))
images = [img, opening, closing]
titles = ['Original','Open','Close']
for i in range(3):
    plt.subplot(1,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i]), plt.xticks([]), plt.yticks([])
plt.show()

### Morphological Gradient, Top Hat, Black Hat
* Morphological Gradient : dilation과 erosion의 차이
* Top Hat : 원본이미지와 open의 차이
* Black Hat : 원본이미지와 close의 차이

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

img = cv2.imread('images/sudoku.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret1, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

kernel = np.ones((4,4),np.uint8)
gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)

# 화면출력
plt.figure(figsize=(18,10))
images = [img, None, None, gradient, tophat, blackhat]
titles = ['Original','', '', 'Gradient','Top Hat', 'Black Hat']
for i in range(6):
    if(images[i] is None): continue
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i]), plt.xticks([]), plt.yticks([])
plt.show()

## 3.3 그래디언트 (Image Gradient)

### Sobel and Scharr Derivatives
* Sobel 연산은 가우시안 스무딩과 미분 연산을 결합한 것으로 노이즈 저항성이 강함
* 가로방향, 세로방향의 미분을 구할 수 있음
* 커널크기를 -1로 주면 Sobel필터가 아닌 Scharr필터를 적용함
* `cv2.Sobel(원본, 색상심도(ddepth), x방향미분차수(dx), y방향미분차수(dy) , 커널크기(ksize))`

### Laplacian Derivatives
* Sobel필터에 라플라시안(dx와 dy가 2인 미분)을 취한 것
* blob(주위의 pixel과 확연한 picel차이를 나타내는 덩어리)검출에 많이 사용됨

### 주의사항
* Sobel 알고리즘은 이미지가 검정->흰색 으로 변할때 양수값을, 흰색->검정 으로 변할때 음수값을 취함
* 이미지의 데이터 유형을 `cv2.CV_U8` 또는 `np.uint8`과 같이 양수만 취급하는 데이터유형의 경우 흰색->검정의 변화를 찾지못함
* 따라서 데이터 부호없는 데이터 유형은 `cv.CV_16S`, `cv.CV_64F`와 같은 상위 데이터 타입으로 변환해서 계산후 후처리를 해야 함

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

img = cv2.imread('images/sudoku.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

laplacian = cv2.Laplacian(img,cv2.CV_64F)
sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)

# 화면출력
plt.figure(figsize=(18,14))
images = [img, laplacian, sobelx, sobely]
titles = ['Original', 'Laplacian','Sobel-X', 'Sobel-Y']
for i in range(4):
    if(images[i] is None): continue
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i]), plt.xticks([]), plt.yticks([])
plt.show()

## 3.4 Canny Edge Detection

### 동작 방식
1. **Noise Reduction** : 5x5의 Gaussian filter 이용한 노이즈 제거
2. **Edge Gradient Detection** : 그래디언트값과 방향을 구함. 그래디언트 값이 급격하게 변하는 곳을 찾음
3. **Non-maximum Suppression** : 경계(edge)가 아닌 영역은 제거 (0으로 만듬)
4. **Hysteresis Thresholding** : 검출한 경계가 진짜 경계인지 임계값으로 걸러냄

### 경계값 검출
* `cv2.Canny(입력이미지, 경계값1, 경계값2[, apertureSize, L2gradient])`
* apertureSize : 그래디언트를 찾기위한 Sobel 커널 크기, 기본값 3
* L2gradient : 그래디언트 측정방법. 기본값 False. 맨하탄 거리측정방식 이용하려면 True로 설정함

In [ ]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('images/phone.jpg', cv2.IMREAD_GRAYSCALE)

edges = cv2.Canny(img,100,200)

plt.figure(figsize=(20,8))
plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()